# Install Necessary Libraries for the Project

In [7]:
# !conda config --append channels pytorch
# !conda install -y datasets accelerate trl rapidfuzz
# !conda install -y conda-forge::bert_score
!conda install -y conda-forge::evaluate

done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /users/0/brogn002/miniconda3/envs/main

  added / updated specs:
    - conda-forge::evaluate


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aiofiles-22.1.0            |  py312h06a4308_0          26 KB
    altair-5.5.0               |  py312h06a4308_0         798 KB
    annotated-types-0.6.0      |  py312h06a4308_0          25 KB
    binaryornot-0.4.4          |     pyhd3eb1b0_1         351 KB
    chardet-4.0.0              |py312h06a4308_1003         217 KB
    click-8.1.7                |  py312h06a4308_0         203 KB
    cookiecutter-2.6.0         |  py312h06a4308_0         139 KB
    evaluate-0.4.3             |     pyh29332c

# Download the dataset from Kaggle and Preprocess it

In [2]:
import os
import kagglehub
import pandas as pd

# Download the dataset from Kaggle
path = kagglehub.dataset_download("shanegerami/ai-vs-human-text")
print("Path to dataset files:", path)

# Convert the .csv dataset file into a Pandas dataframe
df = pd.read_csv(os.path.join(path, "AI_Human.csv"))

# Remove the 'generated' column to save space
df = df.drop(columns=["generated"])

# Display the first few rows of the dataframe
print("Number of rows in the dataframe:", len(df))
df.head()

Path to dataset files: /users/0/brogn002/.cache/kagglehub/datasets/shanegerami/ai-vs-human-text/versions/1
Number of rows in the dataframe: 487235


,text
0,Cars. Cars have been around since they became ...
1,Transportation is a large necessity in most co...
2,"""America's love affair with it's vehicles seem..."
3,How often do you ride in a car? Do you drive a...
4,Cars are a wonderful thing. They are perhaps o...


In [3]:
import random

# Make all of the text lowercase and strip leading and trailing spaces
df["text"] = df["text"].str.lower().str.strip()

# Filter out examples containing non-ASCII characters
def is_valid_example(text):
    if '\\' in r"%r" % text:
        return False
    if '#' in text:
        return False
    try:
        text.encode('ascii')
        return True
    except UnicodeEncodeError:
        return False
df_sample = df[df["text"].apply(is_valid_example)]
print(f"Number of valid ascii examples: {len(df_sample)}")

# Only use 5000 examples from the dataset
df_sample = df_sample.sample(n=5000)
print("Number of rows in the sampled dataframe:", len(df_sample))

# Take a random number of sentences from each sample
for i in range(len(df_sample)):
    df_sample["text"].iloc[i] = (".".join(df_sample["text"].iloc[i].split(".")[:random.randint(2, 6)])) + "."

original_alphabet = 'abcdefghijklmnopqrstuvwxyz'
# Function to generate a random substitution cipher
def generate_cipher(N_letters):
    swap_letters = random.sample(list(original_alphabet), N_letters)
    return ''.join(swap_letters)
# Function to encrypt text using a generated cipher
def encrypt_text(text):
    cipher = generate_cipher(26)
    translation_table = str.maketrans(cipher, "".join(list(cipher)[::-1]))
    encrypted = text.translate(translation_table)
    return encrypted, cipher

# Apply encryption and add new columns
df_sample[['encrypted_text', 'cipher']] = df_sample['text'].apply(encrypt_text).apply(pd.Series)

# Save the processed dataframe to disk
processed_data_path = 'processed_dataset.csv'
df_sample.to_csv(processed_data_path, index=False)
print(f"Processed dataset saved to {processed_data_path}")

# Display the first few rows of the dataframe
df_sample.head()

Number of valid ascii examples: 11801
Number of rows in the sampled dataframe: 5000


/tmp/ipykernel_3105782/2280728614.py:26: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_sample["text"].iloc[i] = (".".join(df_sample["text"].iloc[i].split(".")[:random.randint(2, 6)])) + "."


Processed dataset saved to processed_dataset.csv


,text,encrypted_text,cipher
325334,while the facial action coding system technolo...,snzku onu dpmzpk pmozth mtfzhr wxwouc oumnhtkt...,eikawjcytrfnvqhdgoxmbsplzu
89803,i think it is a good idea because if someone i...,f pyfkn fp fz w xvve fedw cdbwrzd fi zvjdvkd f...,aietmbyxolrzknsuqvghcjpdfw
229309,i believe twat successful people try newt wing...,e diaeisi qplq vhffivvcha wijwai qkm xipq pexz...,gkvclmpjuitdxnbqehowyafsrz
23239,i agree this statement. it is true that young ...,z ktbll gqzc cgkglylfg. zg zc gbdl gqkg mjdft ...,lgmvbjawfqdzsciuhnpkorxyte
404575,after careful research of the options availabl...,iqhsm ximsqvn msesimxt zq hts zghazle iuianidn...,eoajvdpmwhncfqxltyrgbukizs


# Train a model on the Dataset object

In [1]:
%%writefile train_grpo.py

import os
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from transformers import AutoModelForCausalLM
from datasets import Dataset

# Load the preprocessed dataset
processed_data_path = 'processed_dataset.csv'
df = pd.read_csv(processed_data_path)

# Turn the Pandas df into a Dataset object
# Leave 100 examples out of the training set for testing
train_dataset = Dataset.from_pandas(df[100:])

# Create prompts for GRPO Training
def create_prompt(example):
    example["prompt"] = f"""All letters in the following message have been swapped to encrypt the message. Please determine which pairs of letters were swapped and give me the original unencrypted message. Do not output any explanation or additional text beyond the original message. Here is the message to solve with all of the letters swapped: {example["encrypted_text"]} Original: """
    return example
train_dataset = train_dataset.map(create_prompt)
print(train_dataset)

# Determine device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training will run on: {device}")

# Create a unique checkpoint directory for each run using a timestamp
run = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
checkpoint_dir = f'/users/0/brogn002/{run}'
os.makedirs(checkpoint_dir, exist_ok=True)

from trl import GRPOConfig, GRPOTrainer
from rapidfuzz import fuzz

# Define a reward function to use
def reward(completions, **kwargs):
    """
    Computes a similarity score between two strings in the range [0,1].
    """
    original_texts = kwargs["text"]
    rewards = []
    for completion, text in zip(completions, original_texts):
      # Clean up completion to remove any potential formatting
      completion = completion.strip().lower()
      # Do not reward empty strings
      if len(completion) == 0:
            rewards.append(0.0)
            continue
      # Do not reward answers with non-ascii characters
      try:
          completion.encode('ascii')
      except UnicodeEncodeError:
          rewards.append(0.0)
          continue
      # Perfect match gets a full reward
      if completion == text:
          rewards.append(1.0)
          continue
      # Apply RapidFuzz ratio for all cases (handles different lengths well)
      similarity = fuzz.ratio(completion, text) / 100.0
#       # Add additional penalty for length mismatch
#       length_penalty = max(0, 1 - (abs(len(completion) - len(text)) / max(len(text), 1)))
#       # Combined score is a linear combination of similarity and length_penalty
#       final_score = (similarity * 0.5) + (length_penalty * 0.5)
#       rewards.append(final_score)
      rewards.append(similarity)
    return rewards

training_args = GRPOConfig(
    num_train_epochs=2,
    output_dir=checkpoint_dir,
    logging_steps=50,
    per_device_train_batch_size=4,  # Decrease this to lower vram usage
    num_generations=4,  # Decrease this to lower vram usage
    save_strategy="steps",          
    save_steps=500,  
    bf16=True,  # Enable bf16 mixed precision on A100 GPUs
    report_to="wandb",  # Turn on W&B loggin
)

# Path to the model directory
model_dir = '/users/0/brogn002/FOUR_LETTERS_2025-04-13_10-50-32/final_model'
print(os.listdir(model_dir)) # Verify that the folder exists

trainer = GRPOTrainer(
    model=model_dir,
    reward_funcs=reward,
    args=training_args,
    train_dataset=train_dataset,
)

# Train and save the final model
trainer.train()
trainer.save_model(os.path.join(checkpoint_dir, "final_model"))

Overwriting train_grpo.py


In [14]:
!accelerate launch --config_file deepspeed_zero3.yaml train_grpo.py

[2025-04-06 11:28:55,130] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)
df: /users/0/brogn002/.triton/autotune: No such file or directory
Traceback (most recent call last):
  File "/users/0/brogn002/miniconda3/envs/main/bin/accelerate", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/users/0/brogn002/miniconda3/envs/main/lib/python3.12/site-packages/accelerate/commands/accelerate_cli.py", line 48, in main
    args.func(args)
  File "/users/0/brogn002/miniconda3/envs/main/lib/python3.12/site-packages/accelerate/commands/launch.py", line 1177, in launch_command
    deepspeed_launcher(args)
  File "/users/0/brogn002/miniconda3/envs/main/lib/python3.12/site-packages/accelerate/commands/launch.py", line 826, in deepspeed_launcher
    from deepspeed.launcher.runner import DEEPSPEED_ENVIRONMENT_NAME
  File "/users/0/brogn002/miniconda3/envs/main/lib/python3.12/site-packages/deepspeed/__init__.py", line 25, in <module>
    

# Load the Trained Model and Test it Against the Baseline

In [8]:
import os
import evaluate
import bert_score
import statistics
import torch
from tqdm import tqdm
import pandas as pd
import numpy as np
from rapidfuzz import fuzz
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the preprocessed dataset
processed_data_path = 'processed_dataset.csv'
df = pd.read_csv(processed_data_path)

# Turn the Pandas df into a Dataset object
# Only take the first 500 examples
test_dataset = Dataset.from_pandas(df[:500])

# Create prompts
def create_prompt(example):
    example["prompt"] = f"""All letters in the following message have been swapped to encrypt the message. Please determine which pairs of letters were swapped and give me the original unencrypted message. Do not output any explanation or additional text beyond the original message. Here is the message to solve with all of the letters swapped: {example["encrypted_text"]} Original: """
    return example
test_dataset = test_dataset.map(create_prompt)
print(test_dataset)

# Determine device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Evaluation running on: {device}")

# Path to the checkpoints directory
checkpoint_dir = '/users/0/brogn002/FOUR_LETTERS_2025-04-13_10-50-32/final_model'
print(os.listdir(checkpoint_dir)) # Verify that the folder exists

# Load fine-tuned model using local_files_only and from_pretrained
finetuned_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=checkpoint_dir,
    local_files_only=True,
    device_map="auto"  # Support distributed training
)
finetuned_tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=checkpoint_dir,
    local_files_only=True
)

# Load baseline model
baseline_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-4-mini-instruct",
    device_map="auto"  # Support distributed training
)
baseline_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-4-mini-instruct")

finetuned_predictions = []
finetuned_scores = []
baseline_predictions = []
baseline_scores = []
references = []

# Set models to evaluation mode
finetuned_model.eval()
baseline_model.eval()

# Define a reward function to use
def reward(completion, reference):
    """
    Computes a similarity score between two strings in the range [0,1].
    """
    # Clean up completion to remove any potential formatting
    completion = completion.strip().lower()
    # Do not reward empty strings
    if len(completion) == 0:
        return 0.0
    # Do not reward answers with non-ascii characters
    try:
      completion.encode('ascii')
    except UnicodeEncodeError:
      return 0.0
    # Perfect match gets a full reward
    if completion == reference:
      return 1.0
    # Apply RapidFuzz ratio for all cases (handles different lengths well)
    similarity = fuzz.ratio(completion, reference) / 100.0
    return similarity

def test_time_scaling(model, prompt: str, reference: str, tokenizer, num_generations) -> str:
    # Prepare inputs
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Generate output
    outputs = model.generate(
        **inputs,
        num_return_sequences=num_generations,
        num_beams = num_generations,
        max_new_tokens=len(reference),
    )

    # Decode and score outputs
    best_score = 0
    best_output = ""
    for i in range(num_generations):
        decoded = finetuned_tokenizer.decode(outputs[i], skip_special_tokens=True)
        decoded = decoded.replace(prompt, '').strip() # Clean up output (remove prompt)
        score = reward(decoded, reference)
        if score > best_score:
            best_score = score
            best_output = decoded
            
    return best_output, best_score

# Disable gradient computation for inference
with torch.no_grad():
    for example in tqdm(test_dataset, desc="Evaluating Models"):
        encrypted_text = example['encrypted_text']
        original_text = example['text']
        prompt = example['prompt']

        ft_decoded, ft_score = test_time_scaling(finetuned_model, prompt, original_text, finetuned_tokenizer, 4)
        base_decoded, base_score = test_time_scaling(baseline_model, prompt, original_text, baseline_tokenizer, 1)

        finetuned_predictions.append(ft_decoded)
        baseline_predictions.append(base_decoded)
        finetuned_scores.append(ft_score)
        baseline_scores.append(base_score)
        references.append(original_text)

        print(f"\nOriginal Text: {original_text}")
        print(f"\nCipher Text: {encrypted_text}")
        print(f"\nFine-Tuned Model Output: {ft_decoded}")
        print(f"\nBaseline Model Output: {base_decoded}")
        print(f"\nFinetuned Model Reward: {ft_score}")
        print(f"\nBaseline Model Reward: {base_score}")

# Compute overall metrics
bleu = evaluate.load("bleu")
bleu_baseline = bleu.compute(predictions=baseline_predictions, references=references)
bleu_finetuned = bleu.compute(predictions=finetuned_predictions, references=references)

bertscore = evaluate.load("bertscore")
bert_baseline = bertscore.compute(predictions=baseline_predictions, references=references, lang="en")
bert_finetuned = bertscore.compute(predictions=finetuned_predictions, references=references, lang="en")

# Print detailed results
print("\nModel Evaluation Results:")
print(f"Fine-tuned Model - Mean Bleu Score: {statistics.mean(bleu_finetuned['precisions']):.4f}")
print(f"Baseline Model   - Mean Bleu Score: {statistics.mean(bleu_baseline['precisions']):.4f}")
print(f"Fine-tuned Model - Median Bleu Score: {statistics.median(bleu_finetuned['precisions']):.4f}")
print(f"Baseline Model   - Median Bleu Score: {statistics.mean(bleu_baseline['precisions']):.4f}")
print(f"Fine-tuned Model - Bleu Score Std Dev: {statistics.stdev(bleu_finetuned['precisions']):.4f}")
print(f"Baseline Model   - Bleu Score Std Dev: {statistics.stdev(bleu_baseline['precisions']):.4f}")
print(f"Fine-tuned Model - Mean Bert Score: {statistics.mean(bert_finetuned['precision']):.4f}")
print(f"Baseline Model   - Mean Bert Score: {statistics.mean(bert_baseline['precision']):.4f}")
print(f"Fine-tuned Model - Median Bert Score: {statistics.median(bert_finetuned['precision']):.4f}")
print(f"Baseline Model   - Median Bert Score: {statistics.mean(bert_baseline['precision']):.4f}")
print(f"Fine-tuned Model - Bert Score Std Dev: {statistics.stdev(bert_finetuned['precision']):.4f}")
print(f"Baseline Model   - Bert Score Std Dev: {statistics.stdev(bert_baseline['precision']):.4f}")
print(f"Fine-tuned Model - Average Reward: {np.sum(finetuned_scores) / len(finetuned_scores)}")
print(f"Baseline Model   - Average Reward: {np.sum(baseline_scores) / len(baseline_scores)}")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'encrypted_text', 'cipher', 'prompt'],
    num_rows: 100
})
Evaluation running on: cuda
['.ipynb_checkpoints', 'added_tokens.json', 'config.json', 'generation_config.json', 'latest', 'merges.txt', 'model-00001-of-00002.safetensors', 'model-00002-of-00002.safetensors', 'model.safetensors.index.json', 'rng_state_0.pth', 'rng_state_1.pth', 'rng_state_2.pth', 'rng_state_3.pth', 'rng_state_5.pth', 'rng_state_7.pth', 'scheduler.pt', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'trainer_state.json', 'training_args.bin', 'vocab.json', 'zero_to_fp32.py']


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating Models:   1%|▏                     | 1/100 [01:06<1:49:36, 66.43s/it]

...


Original Text: bullying is a widespread problem in today's society and is a major contributor to the psychological distress of many individuals. people may resort to bullying because they are seeking validation, feel insecure, or are trying to cope with their own sense of powerlessness. unfortunately, the consequences of bullying can be severe.

Cipher Text: bullrint is a widmspymad pyoblme in godar's socimgr and is a eajoy congyibugoy go ghm psrcholotical disgymss of eanr indixiduals. pmoplm ear ymsoyg go bullrint bmcausm ghmr aym smmkint xalidagion, fmml insmcuym, oy aym gyrint go copm wigh ghmiy own smnsm of powmylmssnmss. unfoygunagmlr, ghm consmqumncms of bullrint can bm smxmym.

Fine-Tuned Model Output: bullint is a widmspymad pyoblme in godar's socimgr and is a ejoy congyibugoy go ghm psrcholotical disgymss of an individual. poplm ear ymsoyg go bullint because of aym smmkint xalidation, fmml insmcuym, oy aym gyrint go copm wigh ghmiy own smnsm of powmylmssnmss. unfoygunagmlr, t

Evaluating Models:   3%|▋                       | 3/100 [01:27<35:50, 22.17s/it]


Original Text: i remember when i was going through a tough time, i struggled to maintain a good attitude. i didn't want to adhere negative in front of my friends because they hadn't done anything to deserve it..

Cipher Text: i nepepben wher i wag soirs thnoush a toush tipe, i gtnussled to pairtair a sood attitude. i didr't wart to adhene resative ir fnort of py fnierdg bekauge they hadr't dore arythirs to degenve it..

Fine-Tuned Model Output: i nepepben where i wag soirs through a toush type, I gottenled to pairtair a good attitude. I didn't want to adhere resative in fort of py finerdg because they hadn't dore arythirs to degenerate it.

Baseline Model Output: I never knew where I was going through a thick type, I got nudged to pair up a solid attitude. I didn't want to adhere to a reserved interest in spite of my fondness for pie flavor because they hadn't done their research to degenerate it.

Finetuned Model Reward: 0.817258883248731

Baseline Model Reward: 0.7096774193548387


Evaluating Models:   3%|▋                       | 3/100 [01:31<49:18, 30.50s/it]


KeyboardInterrupt: 